In [6]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
import numpy as np
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import statistics 
from statistics import mode
import nltk
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import PorterStemmer 
import sklearn.metrics as metrics
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt

In [28]:
#Load data from json.
#Data has string and target has category.
train_data_data_original_file = []
train_data_target_file = []
test_data_data_original_file = []
test_data_target_file = []

###########################################################
#                                                         #
# CHANGE path_dir to EmotionLines directory               #
# It should end with '/'                                  #
###########################################################
path_dir = 'drive/My Drive/ColabNotebooks/English/EmotionLines/'




dict_emotion_value = { 'joy':0, 'neutral' : 1,'anger':2, 'disgust':3, 'sadness':4, 'surprise':5, 'fear':6, 'non-neutral':7}
with open(path_dir+'Friends/friends_train.json') as train_file:
    train_data = json.load(train_file)
    for episode in train_data:
        for phrase in episode:
            train_data_data_original_file.append(phrase['utterance'])
            train_data_target_file.append(dict_emotion_value[phrase['emotion']])
with open(path_dir + 'Friends/friends_test.json') as test_file:
    test_data = json.load(test_file)
    for episode in test_data:
        for phrase in episode:
            test_data_data_original_file.append(phrase['utterance'])
            test_data_target_file.append(dict_emotion_value[phrase['emotion']])
print(len(train_data_data_original_file))

10561


In [9]:
import pandas as pd


all = pd.DataFrame(list(zip(train_data_data_original_file, train_data_target_file)), columns =['sentence', 'value']) 

train_data_data_original_all = all['sentence']
train_data_target_all = all['value']

In [10]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [11]:
def preprocess(words):
    documents = []
    wnl = WordNetLemmatizer()
    for sen in range(0, len(words)):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(words[sen]))
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)
        # Converting to Lowercase
        document = document.lower()
        # Lemmatization
        document = document.split()
        document = [wnl.lemmatize(word) for word in document]
        result = []
        for word in document: 
            if word not in stop_words:
                result.append(word)
        document = result
        document = ' '.join(document)
        documents.append(document)
    return documents

train_data_data_all = preprocess(train_data_data_original_all)
test_data_data = preprocess(test_data_data_original_file)

In [12]:
def wordsToPadd(words):
  # define documents
  docs = words
  # prepare tokenizer
  t = Tokenizer()
  t.fit_on_texts(docs)
  vocab_size = len(t.word_index) + 1
  # integer encode the documents
  encoded_docs = t.texts_to_sequences(docs)
  # pad documents to a max length of 10 words
  max_length = 10
  padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
  return padded_docs,vocab_size,t
# define class labels
all_labels = array(train_data_target_all)

all_docs,all_vocab,all_t = wordsToPadd(train_data_data_all)

In [13]:
!wget -nc http://nlp.stanford.edu/data/glove.6B.zip
!unzip -n glove*.zip

--2020-06-29 06:32:49--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-29 06:32:50--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-29 06:32:50--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [14]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.300d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [15]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((all_vocab, 300))
for word, i in all_t.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [16]:
# define documents
test_encoded_docs_all = all_t.texts_to_sequences(test_data_data)
max_length = 10
test_padded_docs_all = pad_sequences(test_encoded_docs_all, maxlen=max_length, padding='post')
test_labels = array(test_data_target_file)

In [25]:
from keras.layers import Reshape, Conv2D,Conv1D, GlobalMaxPooling2D,LSTM,MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import optimizers
from keras.optimizers import Adagrad
def imdb_cnn_2():
    sequence_length=10
    embedding_dimension=100
    model = Sequential()

    model.add(Embedding(all_vocab, 300, weights=[embedding_matrix], input_length=10, trainable=False))
    model.add(Conv1D(512,3,padding='same',activation='relu',strides=1))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(Conv1D(64,3,padding='same',activation='relu',strides=1))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dense(32,activation='softmax'))
    
    opt = Adagrad(lr = 0.001)
    model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])
    print(model.summary())

    return model

model = imdb_cnn_2()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 300)           1467900   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 10, 512)           461312    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 5, 512)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 512)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 5, 64)             98368     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 64)               

In [26]:
model.fit(all_docs, all_labels, epochs=50,validation_split=0.2,verbose=2)#, verbose=0)

Train on 8448 samples, validate on 2113 samples
Epoch 1/50
 - 2s - loss: 1.8321 - sparse_categorical_accuracy: 0.4498 - val_loss: 1.6666 - val_sparse_categorical_accuracy: 0.4520
Epoch 2/50
 - 1s - loss: 1.6319 - sparse_categorical_accuracy: 0.4616 - val_loss: 1.6197 - val_sparse_categorical_accuracy: 0.4558
Epoch 3/50
 - 1s - loss: 1.5818 - sparse_categorical_accuracy: 0.4691 - val_loss: 1.5924 - val_sparse_categorical_accuracy: 0.4581
Epoch 4/50
 - 1s - loss: 1.5537 - sparse_categorical_accuracy: 0.4718 - val_loss: 1.5745 - val_sparse_categorical_accuracy: 0.4600
Epoch 5/50
 - 1s - loss: 1.5281 - sparse_categorical_accuracy: 0.4786 - val_loss: 1.5568 - val_sparse_categorical_accuracy: 0.4643
Epoch 6/50
 - 1s - loss: 1.5112 - sparse_categorical_accuracy: 0.4818 - val_loss: 1.5474 - val_sparse_categorical_accuracy: 0.4699
Epoch 7/50
 - 1s - loss: 1.4855 - sparse_categorical_accuracy: 0.4899 - val_loss: 1.5367 - val_sparse_categorical_accuracy: 0.4709
Epoch 8/50
 - 1s - loss: 1.4677 - s

In [23]:
# evaluate the model
loss, accuracy = model.evaluate(test_padded_docs_all, test_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 48.191029


In [22]:
from sklearn.metrics import classification_report
prediction = model.predict(test_padded_docs_all)
predicted_classes = np.argmax(prediction, axis=1)
target_names = ['joy', 'neutral','anger', 'disgust', 'sadness', 'surprise', 'fear', 'non-neutral']
print(prediction)
print(classification_report(predicted_classes,test_data_target_file,zero_division=0,target_names=target_names))

[[4.7903012e-02 5.6206602e-01 1.2819107e-02 ... 5.4554130e-09
  1.1195650e-10 4.0160131e-10]
 [1.1761870e-01 2.5012287e-01 2.9516708e-02 ... 1.0930726e-08
  2.1185843e-10 6.4538314e-10]
 [2.2993119e-02 7.7922696e-01 1.3228792e-02 ... 2.8364326e-07
  1.2011922e-08 4.0843517e-08]
 ...
 [2.9012486e-01 5.3786576e-01 1.7893644e-03 ... 3.4620984e-10
  3.9646203e-12 1.0348255e-11]
 [1.5637996e-02 2.2685820e-01 1.3083334e-01 ... 3.8912140e-07
  1.3828943e-08 3.7705899e-08]
 [5.4549344e-02 4.7969446e-01 8.1696436e-02 ... 1.6966677e-06
  9.6964385e-08 2.6514314e-07]]
              precision    recall  f1-score   support

         joy       0.24      0.42      0.30       171
     neutral       0.81      0.55      0.65      1891
       anger       0.06      0.28      0.10        36
     disgust       0.09      0.40      0.14        15
     sadness       0.19      0.37      0.25        43
    surprise       0.30      0.41      0.34       209
        fear       0.03      0.14      0.05         7
 no